In [90]:
import pandas as pd
import plotly.express as px
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [91]:
#Dataset taken from https://www.kaggle.com/datasets/mohitpoudel/honey-production-in-us-20102021
honeyDF = pd.read_csv("US_honey_production_dataset.csv",
                 sep=",",  # delimiter
                 header=0, # header in first row
                 index_col=0 # ids in first column
                )

In [92]:
honeyDF.head(10)

,state,colony_number,yield_per_colony,productions,stocks,average_price,value_of_prod,year
0,Alabama,9000,54,73000,73000,240.0,1166000,2010
1,Arizona,24000,77,665000,665000,152.0,2809000,2010
2,Arkansas,25000,60,360000,360000,147.0,2205000,2010
3,California,410000,67,6318000,6318000,155.0,42579000,2010
4,Colorado,34000,56,533000,533000,152.0,2894000,2010
5,Florida,200000,69,1794000,1794000,156.0,21528000,2010
6,Georgia,55000,46,152000,152000,167.0,4225000,2010
7,Hawaii,10000,77,239000,239000,275.0,2118000,2010
8,Idaho,97000,27,1179000,1179000,161.0,4217000,2010
9,Illinois,9000,41,92000,92000,278.0,1026000,2010


In [93]:
honeyDF.tail(10)

,state,colony_number,yield_per_colony,productions,stocks,average_price,value_of_prod,year
469,SouthDakota,250000,49,5268000,5268000,2.27,27808000,2021
470,Tennessee,8000,56,81000,81000,5.00,2240000,2021
471,Texas,137000,56,384000,384000,2.30,17646000,2021
472,Utah,31000,33,92000,92000,2.18,2230000,2021
473,Vermont,7000,47,76000,76000,3.28,1079000,2021
474,Virginia,6000,40,79000,79000,8.23,1975000,2021
475,Washington,96000,32,1206000,1206000,2.52,7741000,2021
476,WestVirginia,6000,43,136000,136000,4.80,1238000,2021
477,Wisconsin,42000,47,750000,750000,2.81,5547000,2021
478,Wyoming,38000,58,242000,242000,2.07,4562000,2021


In [94]:
#Checking Pandas dataframe (imported csv) for messiness
honeyDF.describe(include="all") 

,state,colony_number,yield_per_colony,productions,stocks,average_price,value_of_prod,year
count,479,479.000000,479.000000,4.790000e+02,4.790000e+02,479.000000,4.790000e+02,479.000000
unique,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,66578.288100,53.167015,9.127724e+05,9.127724e+05,173.884614,7.832610e+06,2015.505219
std,NaN,99696.073858,17.066853,1.698062e+06,1.698062e+06,147.478891,1.209393e+07,3.457372
min,NaN,3000.000000,27.000000,1.200000e+04,1.200000e+04,1.300000,2.380000e+05,2010.000000
25%,NaN,10000.000000,41.000000,9.000000e+04,9.000000e+04,3.850000,1.611500e+06,2012.500000
50%,NaN,26000.000000,50.000000,2.550000e+05,2.550000e+05,192.000000,3.252000e+06,2016.000000
75%,NaN,77500.000000,62.000000,1.001500e+06,1.001500e+06,243.500000,8.350500e+06,2018.500000


<h1> Replacing Qualitative State Data with Quantitative Data </h1>

In [95]:
#Replacing column data under pd.get_dummies led to 41 separate columns; too messy
#assistance on replacement/dummy variables found via 
#https://www.geeksforgeeks.org/how-to-convert-categorical-variable-to-numeric-in-pandas/#:~:text=Method%201%3A%20Using%20replace(),education%20levels%20into%20numeric%20terms.
honeyDF['state'].replace(['Alabama','Arizona','Arkansas','California','Colorado','Florida','Georgia','Hawaii',
                     'Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine',
                     'Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','NewJersey',
                     'NewMexico','NewYork','NorthCarolina','NorthDakota','Ohio','Oregon','Pennsylvania','SouthDakota',
                     'Tennessee','Texas','Utah','Vermont','Virginia','Washington','WestVirginia','Wisconsin',
                     'Wyoming','SouthCarolina'],
[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,
31,32,33,34,35,36,37,38,39,40], inplace=True)

In [96]:
honeyDF.head(10)

,state,colony_number,yield_per_colony,productions,stocks,average_price,value_of_prod,year
0,0,9000,54,73000,73000,240.0,1166000,2010
1,1,24000,77,665000,665000,152.0,2809000,2010
2,2,25000,60,360000,360000,147.0,2205000,2010
3,3,410000,67,6318000,6318000,155.0,42579000,2010
4,4,34000,56,533000,533000,152.0,2894000,2010
5,5,200000,69,1794000,1794000,156.0,21528000,2010
6,6,55000,46,152000,152000,167.0,4225000,2010
7,7,10000,77,239000,239000,275.0,2118000,2010
8,8,97000,27,1179000,1179000,161.0,4217000,2010
9,9,9000,41,92000,92000,278.0,1026000,2010


In [97]:
#Drop any N/A
honeyDF.dropna()


,state,colony_number,yield_per_colony,productions,stocks,average_price,value_of_prod,year
0,0,9000,54,73000,73000,240.00,1166000,2010
1,1,24000,77,665000,665000,152.00,2809000,2010
2,2,25000,60,360000,360000,147.00,2205000,2010
3,3,410000,67,6318000,6318000,155.00,42579000,2010
4,4,34000,56,533000,533000,152.00,2894000,2010
...,...,...,...,...,...,...,...,...
474,35,6000,40,79000,79000,8.23,1975000,2021
475,36,96000,32,1206000,1206000,2.52,7741000,2021
476,37,6000,43,136000,136000,4.80,1238000,2021
477,38,42000,47,750000,750000,2.81,5547000,2021


In [98]:
honeyDF.describe(include="all") 

,state,colony_number,yield_per_colony,productions,stocks,average_price,value_of_prod,year
count,479.000000,479.000000,479.000000,4.790000e+02,4.790000e+02,479.000000,4.790000e+02,479.000000
mean,19.776618,66578.288100,53.167015,9.127724e+05,9.127724e+05,173.884614,7.832610e+06,2015.505219
std,11.854690,99696.073858,17.066853,1.698062e+06,1.698062e+06,147.478891,1.209393e+07,3.457372
min,0.000000,3000.000000,27.000000,1.200000e+04,1.200000e+04,1.300000,2.380000e+05,2010.000000
25%,9.500000,10000.000000,41.000000,9.000000e+04,9.000000e+04,3.850000,1.611500e+06,2012.500000
50%,19.000000,26000.000000,50.000000,2.550000e+05,2.550000e+05,192.000000,3.252000e+06,2016.000000
75%,30.000000,77500.000000,62.000000,1.001500e+06,1.001500e+06,243.500000,8.350500e+06,2018.500000
max,40.000000,550000.000000,131.000000,1.299500e+07,1.299500e+07,874.000000,8.385900e+07,2021.000000


<h1> Correlation Matrix of Honey Data Table to find Correlation between Column Data </h1>

In [70]:
correlation_matrix = honeyDF.corr()
fig = px.imshow(correlation_matrix, text_auto=True)
fig.show()


In [78]:
# plotting the line chart
fig = px.histogram(honeyDF,x = 'colony_number', y= "value_of_prod")
 
# showing the plot
fig.show()

In [ ]:
# plotting the line chart

fig = px.line(honeyDF, y="sepal_width",x='year',
              color='state')
 
# showing the plot
fig.show()

In [104]:

grouped_multiple = honeyDF.groupby(["year","state"])

grouped_multiple.head()

,state,colony_number,yield_per_colony,productions,stocks,average_price,value_of_prod,year
0,0,9000,54,73000,73000,240.00,1166000,2010
1,1,24000,77,665000,665000,152.00,2809000,2010
2,2,25000,60,360000,360000,147.00,2205000,2010
3,3,410000,67,6318000,6318000,155.00,42579000,2010
4,4,34000,56,533000,533000,152.00,2894000,2010
...,...,...,...,...,...,...,...,...
474,35,6000,40,79000,79000,8.23,1975000,2021
475,36,96000,32,1206000,1206000,2.52,7741000,2021
476,37,6000,43,136000,136000,4.80,1238000,2021
477,38,42000,47,750000,750000,2.81,5547000,2021
